In [11]:
import os
import pandas as pd
import sqlite3
from charset_normalizer import detect

In [13]:
# make a list of files to be added to the sql database 
path = './data/'
files_list = os.listdir(path)
files_list

['departments.csv',
 'dept_emp.csv',
 'dept_manager.csv',
 'employees.csv',
 'salaries.csv',
 'titles.csv']

In [16]:
# determine encoding for each file. 
# This was initially done because the .db file was in files_list resulting in a bug when trying to read in as a csv 
file_encoding_tups = [] 
for file in files_list:
    file_path = path + file
    # open file as readonly binary
    with open(file_path, 'rb') as f:
        # read the file so that the encoding can be detected
        result = detect(f.read())
        # if data exists print encoding andfile in a log string
        if result['encoding'] is not None:
            print('got', result['encoding'], 'as detected encoding for file', file)
        # make a tuple with file name and encoding to add to list 
        file_encoding_tups.append((file, result['encoding']))

got ascii as detected encoding for file departments.csv
got ascii as detected encoding for file dept_emp.csv
got ascii as detected encoding for file dept_manager.csv
got ascii as detected encoding for file employees.csv
got ascii as detected encoding for file salaries.csv
got UTF-8-SIG as detected encoding for file titles.csv


In [19]:
table_df_tup_list = []
for file, encoding in file_encoding_tups:
    # read csv to dataframe with correct encoding
    temp_df = pd.read_csv(f'{path}{file}', encoding=encoding)
    # truncate filename to tablename
    # add table name and df to a list of tuples to add to the sqlite db
    table_df_tup_list.append((file[:-4], temp_df))
# commented bc output was too large
# table_df_tup_list

In [21]:
# Make sqlite db connection.
conn = sqlite3.connect("sql_challenge.sqlite")


In [22]:
for tablename, df in table_df_tup_list:
    df.to_sql(tablename, conn, if_exists='replace')